<a href="https://colab.research.google.com/github/azizmehwish/45_assigment-_by_mehwish/blob/main/crewai_crew.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os


In [2]:
from google.colab import userdata


In [3]:
os.environ['GEMINI_API_KEY'] = userdata.get('GEMINI_API_KEY')

In [4]:
!pip install -Uq crewai crewai-tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.3/265.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.3/211.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 628.3/628.3 kB 30.6 MB/

In [5]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
from crewai.flow.flow import  Flow,start,listen

class MyFlow(Flow):

  @start()
  def function(self):
    print('step1')

  @listen(function)
  def function2(self):
    print('step2')

def kickoff():
 obj=MyFlow()
 obj.kickoff()
print(kickoff())

╭──────────────────────────────────────────────── Flow Execution ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Starting Flow Execution                                                                                        │
│  Name: MyFlow                                                                                                   │
│  ID: c929c903-9ea5-4e96-9e0d-e144483bfea0                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🌊 Flow: MyFlow
    ID: c929c903-9ea5-4e96-9e0d-e144483bfea0
└── 🧠 Starting Flow...

 Flow started with ID: c929c903-9ea5-4e96-9e0d-e144483bfea0


🌊 Flow: MyFlow
    ID: c929c903-9ea5-4e96-9e0d-e144483bfea0
├── 🧠 Starting Flow...
└── 🔄 Running: function

step1


...

🌊 Flow: MyFlow
    ID: c929c903-9ea5-4e96-9e0d-e144483bfea0
├── Flow Method Step
├── ✅ Completed: function
└── 🔄 Running: function2

step2


🌊 Flow: MyFlow
    ID: c929c903-9ea5-4e96-9e0d-e144483bfea0
├── Flow Method Step
├── ✅ Completed: function
└── ✅ Completed: function2

✅ Flow Finished: MyFlow
├── Flow Method Step
├── ✅ Completed: function
└── ✅ Completed: function2

None


In [11]:
from crewai import LLM
llm1= LLM(
    model="gemini/gemini-2.0-flash",
        )
llm1.call("what is the capital of pakistan")

'The capital of Pakistan is **Islamabad**.\n'

In [12]:
google_embedder = {
        "provider": "google",
        "config": {
            "model": "models/text-embedding-004",
            "api_key": "GEMINI_API_KEY",
       }
}

In [13]:
from crewai import Agent, Task, Crew, Process, LLM
from crewai.knowledge.source.string_knowledge_source import StringKnowledgeSource
import os

# Get the GEMINI API key
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

# Create a knowledge source
content = "Users name is mehwish. she is 35 years old and lives in karachi she is teacher."
string_source = StringKnowledgeSource(
    content=content,
)


# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,

)

task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    memory=True,
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    knowledge_sources=[string_source],
    embedder=google_embedder,
)

result = crew.kickoff(inputs={"question": "What city does mehwish live in and how old is she?"})

 
[2025-03-29 07:05:25][ERROR]: Failed to upsert documents: 400 400 POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key. in upsert.
 
[2025-03-29 07:05:25][WARNING]: Failed to init knowledge: 400 400 POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key. in upsert.


╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: cb05dc19-0af2-4d45-b6ea-676eff207f01                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: b6b872c2-8df9-4772-a882-1a1e048a5483
       Status: Executing Task...

ERROR:root:Error during short_term search: 400 400 POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key. in query.
ERROR:root:Error during entities search: 400 400 POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key. in query.


🚀 Crew: crew
└── 📋 Task: b6b872c2-8df9-4772-a882-1a1e048a5483
       Status: Executing Task...
    └── 🤖 Agent: About User
            Status: In Progress

...

ERROR:root:Error during short_term save: 400 400 POST https://generativelanguage.googleapis.com/v1beta/models/text-embedding-004:embedContent?%24alt=json%3Benum-encoding%3Dint: API key not valid. Please pass a valid API key. in add.


🚀 Crew: crew
└── 📋 Task: b6b872c2-8df9-4772-a882-1a1e048a5483
       Status: Executing Task...
    └── 🤖 Agent: About User
            Status: ✅ Completed

🚀 Crew: crew
└── 📋 Task: b6b872c2-8df9-4772-a882-1a1e048a5483
       Assigned to: About User
       Status: ✅ Completed
    └── 🤖 Agent: About User
            Status: ✅ Completed

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: b6b872c2-8df9-4772-a882-1a1e048a5483                                                                     │
│  Agent: About User                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: cb05dc19-0af2-4d45-b6ea-676eff207f01                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [14]:
import os
os.environ['CREWAI_STORAGE_DIR']='/my_crewai'

In [15]:
from crewai import LLM
llm1= LLM(
    model="gemini/gemini-2.0-flash",
        )

In [16]:
from crewai import  Crew, Process
from crewai.memory import LongTermMemory, ShortTermMemory, EntityMemory
from crewai.memory.storage.ltm_sqlite_storage import LTMSQLiteStorage
from crewai.memory.storage.rag_storage import RAGStorage

from typing import List,Optional


# Get the GEMINI API key
GEMINI_API_KEY = os.environ.get("GEMINI_API_KEY")

# Create a knowledge source
content = "Users name is mehwish. she is 35 years old and lives in karachi she is teacher."
string_source = StringKnowledgeSource(
    content=content,
)


# Create an agent with the knowledge store
agent = Agent(
    role="About User",
    goal="You know everything about the user.",
    backstory="""You are a master at understanding people and their preferences.""",
    verbose=True,
    allow_delegation=False,
    llm=llm1,

)

task = Task(
    description="Answer the following questions about the user: {question}",
    expected_output="An answer to the question.",
    agent=agent,
)

crew = Crew(
    agents=[agent],
    tasks=[task],
    verbose=True,
    process=Process.sequential,
    memory=True,
       long_term_memory=LongTermMemory(
           storage=LTMSQLiteStorage(
              db_path="./my_crew2/long_term_storage1.db"
    )
    ),
       short_term_memory=ShortTermMemory(
           storage=RAGStorage(
               embedder_config=google_embedder,
               type="short_term",
               path="./my_crew2/short_term/"
    )
    ),
       entity_memory=EntityMemory(
           storage=RAGStorage(
               embedder_config=google_embedder,
               type="short_term",
               path="./my_crew2/entity/"
         )
    ),




)
